In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf()

In [3]:
sc = SparkContext(conf=conf)

In [4]:
data1 = sc.textFile("matrix1.txt").filter(lambda e: '\t' in e)
data2 = sc.textFile("matrix2.txt").filter(lambda e: '\t' in e)

In [5]:
nRow1 = data1.count()
matrix1 = data1.map(lambda l: l.split(',')).flatMap(lambda entry: [(int(e), int(entry[e])) for e in range(len(entry))])
nCol1 = matrix1.count()/nRow1
matrix1.collect()

[(0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (0, 4),
 (1, 5),
 (2, 6),
 (3, 5),
 (0, 7),
 (1, 8),
 (2, 9),
 (3, 9)]

In [6]:
nRow2 = data2.count()
matrix2 = data2.map(lambda l: l.split(',')).flatMap(lambda entry: [(int(e), int(entry[e])) for e in range(len(entry))])
nCol2 = matrix2.count()/nRow2
matrix2.collect()

[(0, 1), (1, 4), (0, 2), (1, 5), (0, 3), (1, 6), (0, 3), (1, 6)]

In [7]:
mat1 = matrix1.zipWithIndex().map(lambda l: (l[1]//nCol1, l[0][1]))
ma1 = mat1.groupByKey().mapValues(list).map(lambda v: [((v[0], int(i)),v[1]) for i in range(nCol2)])
#m1 = ma1.map(lambda (k, v): [((k, v[i][0]), v[i][1]) for i in range(len(v))])
#m1 = m1.flatMap(lambda l: l).mapValues(lambda v: [v[i][1] for i in range(len(v))])
m1 = ma1.flatMap(lambda l: l)
m1.collect()

[((0, 0), [1, 2, 3, 4]),
 ((0, 1), [1, 2, 3, 4]),
 ((1, 0), [4, 5, 6, 5]),
 ((1, 1), [4, 5, 6, 5]),
 ((2, 0), [7, 8, 9, 9]),
 ((2, 1), [7, 8, 9, 9])]

In [21]:
mat2 = matrix2.groupByKey().mapValues(list)
ma2 = mat2.map(lambda v: [((i, v[0]), v[1]) for i in range(nRow1)])
#m2 = m2.map(value)
m2 = ma2.flatMap(lambda l: l)
m2.collect()

[((0, 0), [1, 2, 3, 3]),
 ((1, 0), [1, 2, 3, 3]),
 ((2, 0), [1, 2, 3, 3]),
 ((0, 1), [4, 5, 6, 6]),
 ((1, 1), [4, 5, 6, 6]),
 ((2, 1), [4, 5, 6, 6])]

In [22]:
matrix = m1.union(m2).groupByKey().mapValues(list)
matrix.collect()

[((2, 0), [[7, 8, 9, 9], [1, 2, 3, 3]]),
 ((0, 0), [[1, 2, 3, 4], [1, 2, 3, 3]]),
 ((1, 1), [[4, 5, 6, 5], [4, 5, 6, 6]]),
 ((0, 1), [[1, 2, 3, 4], [4, 5, 6, 6]]),
 ((1, 0), [[4, 5, 6, 5], [1, 2, 3, 3]]),
 ((2, 1), [[7, 8, 9, 9], [4, 5, 6, 6]])]

In [23]:
def mul(tuple):
    s = 0
    for i in range(len(tuple[1][0])):
        s += tuple[1][0][i] * tuple[1][1][i]
    return (tuple[0], s)

In [25]:
matrix.map(mul).sortByKey().collect()

[((0, 0), 26),
 ((0, 1), 56),
 ((1, 0), 47),
 ((1, 1), 107),
 ((2, 0), 77),
 ((2, 1), 176)]

In [4]:
sc.textFile("matrix1.txt").collect()

[u'1,2,3, 4\t', u'4,5,6, 5\t', u'7,8,9, 9\t']